# Airbnb Pricing Analysis ETL
<b>Updated:</b> 2023-01-07

### Import required libraries

In [1]:
import os                            # for interacting with the operating system
import pandas as pd                  # for manipulating data
from sqlalchemy import create_engine # for creating the connection engine to the database
from mysql import mysql_cnxn         # database credentials

In [2]:
# Get the current path
base_path = os.getcwd()

# Create the directory for the original csv files.
raw_data_path = base_path + "\\Original\\"

# Create the directory for cleaned datasets
cleaned_data_path = base_path + "\\Prepared\\"

# Create the directory for files to be loaded in the database
upload_data_path = base_path + "\\Uploaded\\"

In [3]:
# Create the engone to connect to the MySQL database
eng = create_engine(mysql_cnxn)

### Load the csv files into a Pandas dataframe
Files can be found [here](http://insideairbnb.com/get-the-data).

In [4]:
# Urls for csv files
listings_url = "http://data.insideairbnb.com/united-states/ny/new-york-city/2022-09-07/visualisations/listings.csv"

### Table Schema


| Column name | Description |
| --- | --- |
| id | Listing id |
| name | Name of listing |
| host_id | Host id |
| host_name | Name of host |
| neighbourhood_group | Neighbourhood group the listing is in |
| neighbourhood | Neighbourhood the listing is in |
| latitude | Latitude coordinate of listing location |
| longitude | Longitude coordinate of listing location |
| room_type | Room type of the listing |
| price | Price of the listing |
| minimum_nights | Minimum number of nights stay for listing |
| number_of_reviews | Number of reviews for listing |
| last_review | Date of the latest review |
| reviews_per_month | Number of reviews per month of listing |
| calculated_host_listings_count | Number of listings the host has |
| availability_365 | The availability of the listing in the next 365 days |
| number_of_reviews_ltm | Number of reviews of listing in last 12 months |
| license | If host is licensed |

### Listings

In [5]:
listings_df = pd.read_csv(listings_url)
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,77765,Superior @ Box House,417504,The Box House Hotel,Brooklyn,Greenpoint,40.73777,-73.95366,Hotel room,308,2,42,2022-07-18,0.30,30,217,4,NaN
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64529,-73.97238,Private room,299,30,9,2018-10-19,0.11,9,356,0,NaN
2,45910,Beautiful Queens Brownstone! - 5BR,204539,Mark,Queens,Ridgewood,40.70309,-73.89963,Entire home/apt,425,30,13,2019-11-12,0.10,6,365,0,NaN
3,45935,Room in Beautiful Townhouse.,204586,L,Bronx,Mott Haven,40.80635,-73.92201,Private room,60,30,0,NaN,NaN,1,83,0,NaN
4,45936,Couldn't Be Closer To Columbia Uni,867225,Rahul,Manhattan,Morningside Heights,40.80630,-73.95985,Private room,75,31,135,2022-07-11,0.95,1,219,4,NaN


In [6]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39881 entries, 0 to 39880
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39881 non-null  int64  
 1   name                            39868 non-null  object 
 2   host_id                         39881 non-null  int64  
 3   host_name                       39831 non-null  object 
 4   neighbourhood_group             39881 non-null  object 
 5   neighbourhood                   39881 non-null  object 
 6   latitude                        39881 non-null  float64
 7   longitude                       39881 non-null  float64
 8   room_type                       39881 non-null  object 
 9   price                           39881 non-null  int64  
 10  minimum_nights                  39881 non-null  int64  
 11  number_of_reviews               39881 non-null  int64  
 12  last_review                     

In [7]:
listings_df.to_csv(f"{raw_data_path}raw_listings.csv", index=False)

<b>The 'last_review' column needs to be transformed to type datetime.</b><br>
<b>Null values for 'name' and 'host_name' are unnecessary because of the 'host_id'.</b><br>
<b>Null values for the 'last_review' and 'reviews_per_month' are features I wish to keep to check against 'number_of_reviews'.</b><br>
<b>Optimize the data types before uploading.</b>

In [8]:
# Convert 'last_review' to datetime data type.
listings_df["last_review"] = pd.to_datetime(listings_df["last_review"])
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39881 entries, 0 to 39880
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              39881 non-null  int64         
 1   name                            39868 non-null  object        
 2   host_id                         39881 non-null  int64         
 3   host_name                       39831 non-null  object        
 4   neighbourhood_group             39881 non-null  object        
 5   neighbourhood                   39881 non-null  object        
 6   latitude                        39881 non-null  float64       
 7   longitude                       39881 non-null  float64       
 8   room_type                       39881 non-null  object        
 9   price                           39881 non-null  int64         
 10  minimum_nights                  39881 non-null  int64         
 11  nu

In [9]:
# Convert remaining data types
listings_df = listings_df.convert_dtypes()
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39881 entries, 0 to 39880
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              39881 non-null  Int64         
 1   name                            39868 non-null  string        
 2   host_id                         39881 non-null  Int64         
 3   host_name                       39831 non-null  string        
 4   neighbourhood_group             39881 non-null  string        
 5   neighbourhood                   39881 non-null  string        
 6   latitude                        39881 non-null  Float64       
 7   longitude                       39881 non-null  Float64       
 8   room_type                       39881 non-null  string        
 9   price                           39881 non-null  Int64         
 10  minimum_nights                  39881 non-null  Int64         
 11  nu

In [10]:
# Check summary statistics before uploading
listings_df.describe(include="all", datetime_is_numeric=True)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
count,3.988100e+04,39868,3.988100e+04,39831,39881,39881,39881.000000,39881.000000,39881,39881.000000,39881.000000,39881.000000,31519,31519.000000,39881.000000,39881.000000,39881.000000,5
unique,NaN,38696,NaN,9478,5,244,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
top,NaN,Water View King Bed Hotel Room,NaN,Blueground,Manhattan,Bedford-Stuyvesant,NaN,NaN,Entire home/apt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41662/AL
freq,NaN,30,NaN,453,16847,2779,NaN,NaN,22761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,1.315489e+14,NaN,1.313420e+08,NaN,NaN,NaN,40.728870,-73.945665,NaN,197.547228,19.131742,26.690855,2021-03-23 16:19:17.701703680,1.244733,16.931396,131.535669,7.754344,NaN
min,2.539000e+03,NaN,2.438000e+03,NaN,NaN,NaN,40.500314,-74.269520,NaN,0.000000,1.000000,0.000000,2011-05-12 00:00:00,0.010000,1.000000,0.000000,0.000000,NaN
25%,1.633197e+07,NaN,1.363938e+07,NaN,NaN,NaN,40.687760,-73.983340,NaN,80.000000,2.000000,1.000000,2020-01-04 00:00:00,0.140000,1.000000,0.000000,0.000000,NaN
50%,3.823683e+07,NaN,5.974663e+07,NaN,NaN,NaN,40.724545,-73.953710,NaN,130.000000,14.000000,5.000000,2022-06-21 00:00:00,0.560000,1.000000,75.000000,1.000000,NaN
75%,5.255780e+07,NaN,2.233746e+08,NaN,NaN,NaN,40.763200,-73.925600,NaN,219.000000,30.000000,25.000000,2022-08-21 00:00:00,1.790000,4.000000,277.000000,7.000000,NaN
max,7.098549e+17,NaN,4.782606e+08,NaN,NaN,NaN,40.928810,-73.690060,NaN,16500.000000,1250.000000,1480.000000,2022-09-07 00:00:00,123.000000,453.000000,365.000000,949.000000,NaN


<b>There appears to be record(s) that have no price (0) value.</b>

### Top Categorical Variables
*Frequency in parenthesis.*
<br>
<b>Name:</b> Water View King Bed Hotel Room (30)<br>
<b>Host Name:</b> Blueground (453)<br>
<b>Neighborhood Group:</b> Manhattan (16847)<br>
<b>Neighborhood:</b> Bedford-Stuyvesant (2779)<br>
<b>Room Type:</b> Entire home/apt (22761)

In [11]:
# Change the 'neighbourhood_group' and 'neighbourhood' column names
listings_df.rename(columns={"neighbourhood_group": "neighborhood_group", "neighbourhood": "neighborhood",
                            "calculated_host_listings_count": "total_host_listings"}, inplace=True)
listings_df.head()

,id,name,host_id,host_name,neighborhood_group,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,77765,Superior @ Box House,417504,The Box House Hotel,Brooklyn,Greenpoint,40.73777,-73.95366,Hotel room,308,2,42,2022-07-18,0.3,30,217,4,<NA>
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64529,-73.97238,Private room,299,30,9,2018-10-19,0.11,9,356,0,<NA>
2,45910,Beautiful Queens Brownstone! - 5BR,204539,Mark,Queens,Ridgewood,40.70309,-73.89963,Entire home/apt,425,30,13,2019-11-12,0.1,6,365,0,<NA>
3,45935,Room in Beautiful Townhouse.,204586,L,Bronx,Mott Haven,40.80635,-73.92201,Private room,60,30,0,NaT,<NA>,1,83,0,<NA>
4,45936,Couldn't Be Closer To Columbia Uni,867225,Rahul,Manhattan,Morningside Heights,40.8063,-73.95985,Private room,75,31,135,2022-07-11,0.95,1,219,4,<NA>


### Remove records where the price is 0.
These records will not provide any insight into how the features affect price.

In [12]:
zero_prices = listings_df[listings_df["price"] == 0]

In [13]:
# Save deleted records to the Error folder
zero_prices.to_csv(f"{base_path}\\Errors\\zero_prices.csv", index=False)

In [14]:
listings_df = listings_df[listings_df.price != 0]

In [15]:
# Save dataframe 
listings_df.to_csv(f"{cleaned_data_path}listings_clean.csv", index=False)

### Removing columns that won't be included during analysis
These columns contained null values and while I can delete the rows and keep the columns, I concluded that these features were not needed for analysis.

In [ ]:
listings_df.drop(columns=["name", "host_name", "last_review", "reviews_per_month", "license"])

In [ ]:
listings_df.to_csv(f"{upload_data_path}listings.csv", index=False)

In [ ]:
# Upload dataframe to MySQL database
listings_df.to_sql(name="listings", con=eng, if_exists="replace", index=False)